<a href="https://colab.research.google.com/github/issei/DaedalusForge/blob/main/notebooks/automations/selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construindo um Agente de Automação Web com LangGraph e Selenium

Neste notebook, vamos aprender a construir um agente de IA autônomo capaz de navegar e interagir com a web. Faremos isso combinando três tecnologias poderosas:

Selenium: A principal ferramenta para automação de navegadores. Serão as "mãos" do nosso agente.

LangGraph: Um framework para construir agentes de IA robustos e cíclicos, orquestrando múltiplos passos. Será o "esqueleto" do nosso agente.

Modelos de Linguagem (LLMs): Como o Google Gemini, que fornecerá a capacidade de raciocínio. Será o "cérebro" do nosso agente.

Nosso objetivo é criar uma ferramenta de automação que segue um padrão similar ao UTCP (Universal Task-Centric Protocol), permitindo que o agente execute tarefas web complexas a partir de instruções em linguagem natural, sem precisar conhecer os detalhes do Selenium.

In [1]:
# Esta célula prepara o ambiente instalando as bibliotecas necessárias.
%pip install -q selenium langgraph langchain langchain_gemini langchain_community langchain_google_genai

In [2]:
import google.generativeai as genai
from google.colab import userdata

# Recupere a GOOGLE_API_KEY dos segredos do Colab
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure a API do Google Generative AI
genai.configure(api_key=GOOGLE_API_KEY)

print("API do Google Generative AI configurada com sucesso!")

API do Google Generative AI configurada com sucesso!


## Passo 2: Criando a Ferramenta SeleniumUTCPTool

Aqui está o coração da nossa integração. Vamos criar uma classe Python que encapsula as funcionalidades do Selenium. Esta classe irá:

1. Iniciar e gerenciar uma instância do Chrome WebDriver em modo headless.
2. Expor um único método `@tool` chamado `execute_task`. Este método receberá um comando em formato JSON (inspirado no UTCP) e o traduzirá em ações do Selenium, como navegar, clicar, ler ou digitar.
3. Retornar um resultado também em JSON, informando o status da operação (sucesso ou erro) e, se aplicável, dados extraídos da página (como texto ou atributos de elementos).

Essa arquitetura desacopla o "cérebro" do agente da ferramenta de execução. O agente só precisa saber construir o JSON, e não como o Selenium funciona.

In [3]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException

# Decorador para integrar com LangChain
def tool(func):
    func.__dict__['is_tool'] = True
    return func

class SeleniumUTCPTool:
    """
    Uma ferramenta que utiliza Selenium para executar comandos de automação web
    baseados em um formato JSON inspirado no UTCP.
    """
    def __init__(self):
        """Inicializa o WebDriver do Chrome em modo headless."""
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # Adicionar outras opções úteis, como user-agent
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')


        try:
            # Configura o Selenium Manager para baixar e gerenciar o driver
            service = Service()
            self.driver = webdriver.Chrome(service=service, options=options)
            self.driver.implicitly_wait(5) # Espera implícita para encontrar elementos
            print("WebDriver inicializado com sucesso.")
        except WebDriverException as e:
            print(f"Erro ao inicializar o WebDriver: {e}")
            self.driver = None # Garante que o driver seja None em caso de erro

    def _navigate_to_url(self, url):
        """Navega para uma URL específica."""
        try:
            self.driver.get(url)
            return {"status": "success", "message": f"Navegou para: {url}"}
        except WebDriverException as e:
            return {"status": "error", "message": f"Erro ao navegar para {url}: {e}"}

    def _find_element(self, by, value, wait_time=10):
        """Encontra um elemento usando o localizador especificado, esperando por sua presença."""
        try:
            wait = WebDriverWait(self.driver, wait_time)
            if by.lower() == 'id':
                element = wait.until(EC.presence_of_element_located((By.ID, value)))
            elif by.lower() == 'name':
                element = wait.until(EC.presence_of_element_located((By.NAME, value)))
            elif by.lower() == 'xpath':
                 element = wait.until(EC.presence_of_element_located((By.XPATH, value)))
            elif by.lower() == 'css_selector':
                 element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, value)))
            elif by.lower() == 'link_text':
                 element = wait.until(EC.presence_of_element_located((By.LINK_TEXT, value)))
            elif by.lower() == 'partial_link_text':
                 element = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, value)))
            elif by.lower() == 'tag_name':
                 element = wait.until(EC.presence_of_element_located((By.TAG_NAME, value)))
            elif by.lower() == 'class_name':
                 element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, value)))
            else:
                return {"status": "error", "message": f"Localizador desconhecido: {by}"}
            return {"status": "success", "element": element}
        except TimeoutException:
            return {"status": "error", "message": f"Tempo esgotado esperando pelo elemento {value} usando {by}"}
        except NoSuchElementException:
             return {"status": "error", "message": f"Elemento não encontrado: {value} usando {by}"}
        except WebDriverException as e:
            return {"status": "error", "message": f"Erro ao encontrar elemento {value} usando {by}: {e}"}


    def _find_element_and_click(self, by, value):
        """Encontra um elemento e clica nele."""
        find_result = self._find_element(by, value)
        if find_result["status"] == "success":
            try:
                find_result["element"].click()
                return {"status": "success", "message": f"Clicou no elemento: {value} usando {by}"}
            except WebDriverException as e:
                return {"status": "error", "message": f"Erro ao clicar no elemento {value} usando {by}: {e}"}
        else:
            return find_result # Retorna o erro de _find_element


    def _find_element_and_type(self, by, value, text):
        """Encontra um campo de texto e digita um valor."""
        find_result = self._find_element(by, value)
        if find_result["status"] == "success":
            try:
                find_result["element"].send_keys(text)
                return {"status": "success", "message": f"Digitou '{text}' no elemento: {value} usando {by}"}
            except WebDriverException as e:
                return {"status": "error", "message": f"Erro ao digitar no elemento {value} usando {by}: {e}"}
        else:
            return find_result # Retorna o erro de _find_element


    def _get_element_text(self, by, value):
        """Encontra um elemento e retorna seu texto."""
        find_result = self._find_element(by, value)
        if find_result["status"] == "success":
            try:
                text = find_result["element"].text
                return {"status": "success", "text": text, "message": f"Texto do elemento {value} usando {by} obtido com sucesso."}
            except WebDriverException as e:
                 return {"status": "error", "message": f"Erro ao obter texto do elemento {value} usando {by}: {e}"}
        else:
            return find_result # Retorna o erro de _find_element

    def _get_element_attribute(self, by, value, attribute_name):
        """Encontra um elemento e retorna o valor de um atributo."""
        find_result = self._find_element(by, value)
        if find_result["status"] == "success":
            try:
                attribute_value = find_result["element"].get_attribute(attribute_name)
                return {"status": "success", "attribute": attribute_name, "value": attribute_value, "message": f"Atributo '{attribute_name}' do elemento {value} usando {by} obtido com sucesso."}
            except WebDriverException as e:
                 return {"status": "error", "message": f"Erro ao obter atributo '{attribute_name}' do elemento {value} usando {by}: {e}"}
        else:
            return find_result # Retorna o erro de _find_element


    def _get_current_url(self):
        """Retorna a URL atual da página."""
        try:
            current_url = self.driver.current_url
            return {"status": "success", "url": current_url, "message": "URL atual obtida com sucesso."}
        except WebDriverException as e:
            return {"status": "error", "message": f"Erro ao obter a URL atual: {e}"}

    def _quit_driver(self):
        """Fecha o navegador e encerra a sessão do WebDriver."""
        if self.driver:
            try:
                self.driver.quit()
                self.driver = None
                return {"status": "success", "message": "WebDriver encerrado com sucesso."}
            except WebDriverException as e:
                return {"status": "error", "message": f"Erro ao encerrar o WebDriver: {e}"}
        else:
             return {"status": "success", "message": "WebDriver já estava encerrado."}


    @tool
    def execute_task(self, json_command: str) -> str:
        """
        Executa uma tarefa de automação web baseada em um comando JSON.

        O JSON de entrada deve ter a seguinte estrutura:
        {
          "action": "navigate" | "click" | "type" | "read_text" | "read_attribute" | "get_url" | "quit",
          "url": "http://example.com" (obrigatório para "navigate"),
          "by": "id" | "name" | "xpath" | "css_selector" | "link_text" | "partial_link_text" | "tag_name" | "class_name" (obrigatório para "click", "type", "read_text", "read_attribute"),
          "value": "valor_do_localizador" (obrigatório para "click", "type", "read_text", "read_attribute"),
          "text": "texto_a_digitar" (obrigatório para "type"),
          "attribute": "nome_do_atributo" (obrigatório para "read_attribute")
        }

        Retorna um JSON com o status da operação e, se aplicável, dados resultantes.
        Exemplos de retorno:
        {"status": "success", "message": "..."}
        {"status": "error", "message": "..."}
        {"status": "success", "text": "texto_extraido", "message": "..."}
        {"status": "success", "attribute": "href", "value": "url_do_link", "message": "..."}
        {"status": "success", "url": "url_atual", "message": "..."}
        """
        if not self.driver:
            return json.dumps({"status": "error", "message": "WebDriver não inicializado. Verifique o log de inicialização."})

        try:
            command = json.loads(json_command)
            action = command.get("action")

            if action == "navigate":
                url = command.get("url")
                if not url:
                    return json.dumps({"status": "error", "message": "URL é obrigatória para a ação 'navigate'."})
                result = self._navigate_to_url(url)
            elif action == "click":
                by = command.get("by")
                value = command.get("value")
                if not by or not value:
                     return json.dumps({"status": "error", "message": "'by' e 'value' são obrigatórios para a ação 'click'."})
                result = self._find_element_and_click(by, value)
            elif action == "type":
                by = command.get("by")
                value = command.get("value")
                text = command.get("text")
                if not by or not value or text is None:
                    return json.dumps({"status": "error", "message": "'by', 'value' e 'text' são obrigatórios para a ação 'type'."})
                result = self._find_element_and_type(by, value, text)
            elif action == "read_text":
                by = command.get("by")
                value = command.get("value")
                if not by or not value:
                    return json.dumps({"status": "error", "message": "'by' e 'value' são obrigatórios para a ação 'read_text'."})
                result = self._get_element_text(by, value)
            elif action == "read_attribute":
                by = command.get("by")
                value = command.get("value")
                attribute_name = command.get("attribute")
                if not by or not value or not attribute_name:
                    return json.dumps({"status": "error", "message": "'by', 'value' e 'attribute' são obrigatórios para a ação 'read_attribute'."})
                result = self._get_element_attribute(by, value, attribute_name)
            elif action == "get_url":
                result = self._get_current_url()
            elif action == "quit":
                 result = self._quit_driver()
            else:
                result = {"status": "error", "message": f"Ação desconhecida: {action}"}

            return json.dumps(result)

        except json.JSONDecodeError:
            return json.dumps({"status": "error", "message": "Comando JSON inválido."})
        except Exception as e:
            return json.dumps({"status": "error", "message": f"Erro inesperado durante a execução da tarefa: {e}"})

## Passo 3: Construindo o Grafo do Agente com LangGraph e Gemini

Agora que temos as "mãos" (SeleniumUTCPTool), vamos construir a estrutura do nosso agente usando LangGraph. A principal mudança aqui é que usaremos o ChatGoogleGenerativeAI como o "cérebro".
O fluxo de trabalho será um ciclo:

1.  **Agente (call_model)**: O Gemini recebe a pergunta e o histórico, e decide se deve responder ao usuário ou usar uma ferramenta. Se usar uma ferramenta, ele gera o comando JSON para o `execute_task`.
2.  **Execução da Ferramenta (call_tool)**: O comando JSON é passado para nossa ferramenta Selenium, que executa a ação no navegador.
3.  **Retorno**: O resultado da ferramenta é enviado de volta para o agente.

O ciclo se repete até que o Gemini decida que tem informação suficiente para dar a resposta final.

In [4]:
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import BaseMessage
from langgraph.prebuilt import ToolNode # Importe ToolNode conforme solicitado
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END

# Instancia a ferramenta SeleniumUTCPTool
# Assumimos que a classe SeleniumUTCPTool já foi definida em uma célula anterior
selenium_tool_instance = SeleniumUTCPTool()

# Cria uma lista de ferramentas a serem usadas pelo agente
tools = [selenium_tool_instance.execute_task]
# Usa ToolNode para executar as ferramentas
tool_node = ToolNode(tools)


# Define o modelo Gemini a ser usado e associa as ferramentas a ele
# Certifique-se de ter a GOOGLE_API_KEY configurada no seu ambiente
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, google_api_key=GOOGLE_API_KEY) # Explicitly pass API key
model = model.bind_tools(tools)

# Define o estado do agente. É simplesmente uma lista de mensagens.
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

# Define o nó que chama o modelo
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

# Define a lógica condicional para decidir o próximo passo
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # Se a última mensagem tem chamadas de ferramenta, execute a ferramenta
    if last_message.tool_calls:
        return "action" # Se 'continue', vai para 'action' (o ToolNode)
    # Caso contrário, o agente terminou (gerou uma resposta final)
    else:
        return END # Se 'end', termina o grafo

# Constrói o grafo
graph = StateGraph(AgentState)

# Adiciona os nós
graph.add_node("agent", call_model)
graph.add_node("action", tool_node) # Usa o ToolNode como o nó de ação

# Define o ponto de entrada
graph.set_entry_point("agent")

# Define as bordas (transições entre nós)
graph.add_conditional_edges(
    "agent", # De 'agent'
    should_continue # Usa a função condicional para decidir se vai para 'action' ou termina
)

# De 'action' (o ToolNode), sempre volta para 'agent' para que o modelo processe o resultado da ferramenta
graph.add_edge('action', 'agent')

# Compila o grafo
app = graph.compile()

print("Grafo do agente LangGraph compilado com sucesso!")

WebDriver inicializado com sucesso.
Grafo do agente LangGraph compilado com sucesso!


## Passo 4: Executando o Agente e Navegando na Web!

Tudo pronto! Agora vamos testar nosso agente com uma tarefa real. Vamos pedir a ele para navegar até o site do Selenium e extrair uma informação.
Observe a saída da célula abaixo. Você verá o "raciocínio" do agente passo a passo: a primeira chamada de ferramenta para navegar, a segunda para encontrar o elemento e ler o texto, e finalmente a resposta consolidada.

In [5]:
from langchain_core.messages import HumanMessage

# Instancia a ferramenta SeleniumUTCPTool antes de executar a tarefa
selenium_tool_instance = SeleniumUTCPTool()

# Define a tarefa para o agente
inputs = {"messages": [HumanMessage(content="Navegue até https://uol.com.br, procure por 'LangChain' e me diga o conteúdo")]}

# Executa o agente e imprime a saída de cada passo
try:
    for output in app.stream(inputs):
        # stream() yields dicts with the node name as the key
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")

finally:
    # Garante que o WebDriver seja fechado no final
    selenium_tool_instance._quit_driver()
    print("WebDriver encerrado.")

WebDriver inicializado com sucesso.
Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'execute_task', 'arguments': '{"json_command": "{\\"action\\": \\"navigate\\", \\"url\\": \\"https://uol.com.br\\"}"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--239bac4b-82e0-4812-8315-936b5342d882-0', tool_calls=[{'name': 'execute_task', 'args': {'json_command': '{"action": "navigate", "url": "https://uol.com.br"}'}, 'id': '02c938dc-c4ee-45ec-b5b8-344b5dcad6fb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 426, 'output_tokens': 448, 'total_tokens': 874, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 412}})]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content='{"status": "success", "message": "Navegou para: https://uol.com.br"}', name='execute_task'